# 🚀 Freqtrade 策略测试器（通用教程）

本 Notebook 用于通用地测试任意 Freqtrade 策略，支持：

- ✅ 验证策略基础功能（加载、参数、时间周期等）
- ✅ 测试自定义 Hyperopt Loss（可选）
- ✅ 检查数据加载状态（本地 feather 数据）
- ✅ 生成测试报告与下一步建议

适用于：新策略调试、参数优化前检查、快速多策略对比。


## 🔧 使用说明

1. 修改下方“用户配置参数”，可指定任意策略与配置文件。
2. 依次运行每个单元格。
3. 根据“测试结果汇总”中的建议进行下一步（回测/优化）。


In [7]:
# ==============================
# 🔧 用户配置参数（按需修改）
# ==============================

# 策略类名（与 user_data/strategies/<文件名>.py 内类名一致）
STRATEGY_NAME = "DoubleMAStrategy"  # 例："SampleStrategy"、"DoubleMAStrategy"

# 配置文件路径（建议使用你实际的配置）
CONFIG_FILE = "../user_data/config_double_ma.json"

# 自定义 Hyperopt Loss 类名（可选，无则留空字符串）
HYPEROPT_LOSS_NAME = "DoubleMAHyperOptLoss"  # 例："SampleHyperOptLoss"，无则设为 ""

# 测试模式："basic" | "loss" | "data" | "full"
TEST_MODE = "full"

# ==============================
# 🔧 环境初始化
# ==============================

import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime

print("✅ 环境初始化完成")
print(f"📁 工作目录: {project_root}")
print(f"🎯 策略: {STRATEGY_NAME}")
print(f"⚙️ 配置: {CONFIG_FILE}")


✅ 环境初始化完成
📁 工作目录: /Users/apple/code/freqtrade/scripts
🎯 策略: DoubleMAStrategy
⚙️ 配置: ../user_data/config_double_ma.json


## 🔍 测试一：策略基础功能


In [8]:
def test_strategy_basic_functionality():
    print("🔍 测试策略基础功能...")
    try:
        from freqtrade.resolvers import StrategyResolver
        from freqtrade.configuration import Configuration

        config = Configuration.from_files([CONFIG_FILE])
        # 覆盖策略名称（以防配置中不同）
        config["strategy"] = STRATEGY_NAME

        strat = StrategyResolver.load_strategy(config)

        print("✅ 策略加载成功")
        print(f"   📄 策略类: {strat.__class__.__name__}")
        print(f"   ⏰ 时间周期: {getattr(strat, 'timeframe', 'N/A')}")
        print(f"   📈 允许做空: {getattr(strat, 'can_short', 'N/A')}")
        print(f"   🛡️ 止损: {getattr(strat, 'stoploss', 'N/A')}")
        print(f"   🎯 ROI: {getattr(strat, 'minimal_roi', 'N/A')}")

        # 列出可优化参数
        print("\n📊 可优化参数:")
        count = 0
        for name in dir(strat):
            obj = getattr(strat, name)
            if hasattr(obj, "optimize") and getattr(obj, "optimize"):
                count += 1
                rng = []
                for r in ("low", "high", "choices"):
                    if hasattr(obj, r):
                        rng.append(f"{r}={getattr(obj, r)}")
                print(f"   🔧 {name}: value={getattr(obj, 'value', None)}  ({', '.join(rng)})")
        if count == 0:
            print("   ⚠️ 未发现可优化参数（正常情况：若策略未启用Parameter）")

        return True
    except Exception as e:
        print(f"❌ 策略加载失败: {e}")
        print("   💡 检查配置文件路径 / 策略类名 / 依赖环境")
        return False

if TEST_MODE in ("basic", "full"):
    _ok_basic = test_strategy_basic_functionality()
else:
    print("⏭️ 跳过基础功能测试")
    _ok_basic = None


🔍 测试策略基础功能...


2025-09-20 12:16:37,353 - freqtrade.configuration.load_config - INFO - Using config: ../user_data/config_double_ma.json ...

2025-09-20 12:16:37,362 - freqtrade.loggers - INFO - Enabling colorized output.

2025-09-20 12:16:37,363 - freqtrade.loggers - INFO - Logfile configured

2025-09-20 12:16:37,364 - freqtrade.loggers - INFO - Verbosity set to 0

❌ 策略加载失败: Directory `user_data` does not exist. Please use `freqtrade create-userdir` to create a user directory
   💡 检查配置文件路径 / 策略类名 / 依赖环境


## 📊 测试二：Hyperopt Loss（可选）


In [ ]:
def test_hyperopt_loss():
    if not HYPEROPT_LOSS_NAME:
        print("⏭️ 未配置自定义Loss，跳过")
        return None
    print("📊 测试Hyperopt Loss...")
    try:
        module = __import__(f"user_data.hyperopts.{HYPEROPT_LOSS_NAME}", fromlist=[HYPEROPT_LOSS_NAME])
        LossCls = getattr(module, HYPEROPT_LOSS_NAME)
        print(f"✅ 加载成功: {LossCls.__name__}")

        mock = pd.DataFrame({
            "profit_ratio": [0.02, -0.01, 0.03, 0.01, -0.005, 0.025],
            "trade_duration": [120, 180, 90, 240, 60, 150],
        })
        val = LossCls.hyperopt_loss_function(
            results=mock,
            trade_count=len(mock),
            min_date=pd.Timestamp("2024-01-01"),
            max_date=pd.Timestamp("2024-01-10"),
            config={},
            processed={},
        )
        print(f"🎯 Loss值: {val:.6f}（越小越好）")
        return True
    except Exception as e:
        print(f"❌ Loss测试失败: {e}")
        return False

if TEST_MODE in ("loss", "full"):
    _ok_loss = test_hyperopt_loss()
else:
    print("⏭️ 跳过Loss测试")
    _ok_loss = None


## 📈 测试三：数据加载检查


In [ ]:
def test_data_loading():
    print("📈 检查数据加载...")
    data_dir = project_root / "user_data" / "data"
    if not data_dir.exists():
        print("❌ 未找到数据目录 user_data/data")
        print("   💡 运行: freqtrade download-data --timeframe 1h --timerange 20230101-20231231")
        return False, 0

    files = list(data_dir.glob("**/*.feather"))
    if not files:
        print("❌ 未找到 feather 数据文件")
        return False, 0

    total = 0
    for fp in files[:10]:  # 采样检查
        try:
            df = pd.read_feather(fp)
            total += len(df)
        except Exception as e:
            print(f"⚠️ 文件读取失败 {fp.name}: {e}")

    print(f"✅ 数据文件: {len(files)} 个，采样合计 {total} 行")
    return True, len(files)

if TEST_MODE in ("data", "full"):
    _ok_data, _files = test_data_loading()
else:
    print("⏭️ 跳过数据检查")
    _ok_data, _files = None, 0


## 🧾 测试结果汇总与下一步


In [ ]:
def summarize():
    print("\n===== 测试汇总 =====")
    passed = 0
    total = 0
    if TEST_MODE in ("basic", "full"):
        total += 1
        print(f"基础功能: {'✅' if _ok_basic else '❌'}")
        passed += 1 if _ok_basic else 0
    if TEST_MODE in ("loss", "full"):
        total += 1
        print(f"Loss测试: {'✅' if _ok_loss else '❌' if _ok_loss is False else '⏭️'}")
        passed += 1 if _ok_loss else 0
    if TEST_MODE in ("data", "full"):
        total += 1
        print(f"数据检查: {'✅' if _ok_data else '❌' if _ok_data is False else '⏭️'}")
        passed += 1 if _ok_data else 0
    if total:
        print(f"结果: {passed}/{total} 通过")
        if passed == total:
            print("🎉 可以进行回测 / 超参优化啦！")
        else:
            print("⚠️ 建议先修复失败项再继续。")

summarize()
